# Dive Prediction - Data

*Predicting Seabird Diving Behaviour from GPS data*

This notebook prepares and formats a dataset of **74 foraging trips of seabirds** (*Sula Variegata*) with both GPS and Time Depth Recorder (TDR) tracking data. Data has been collected on **Ilha Pescadores, Peru** between 2009 and 2013.

More precisely, data have been prepared and selected as follow:

* Trajectories with raw temporal sampling at 1s only
* Gaps have been interpolated linearly (yet there are shorter than 10s)
* Maximal step speed is inferior to 50 m.s-1
* Birds never stay static longer than 10 minutes
* Number of dive in each trajectory is within the following range 1-99

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.trip import * 

In [ ]:
 np.random.seed(1)

## load raw data 

In [ ]:
# connect to database
conn = sqlite3.connect('/home/amdroy/MEGA/DATA/seabirdbank.db')

In [ ]:
request = "SELECT trip, datetime, lon, lat, pressure, fastlog, gaps, step_speed, step_direction FROM dive \
            INNER JOIN trip ON dive.trip = trip.id \
            INNER JOIN bird ON trip.bird = bird.id \
            WHERE bird.species = 'SV' \
            ORDER BY trip, datetime"

data = pd.read_sql_query(request, conn)
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S')
data['dive'] = 0

# convert to boolean
data['gaps'] = (data['gaps']=='TRUE')
data['fastlog'] = (data['fastlog']=='TRUE')

data

## filter data

In [ ]:
check = pd.read_csv("/home/amdroy/MEGA/DATA/FORMAT/dive_check.csv", sep = ';')
check = check[check.trip.isin(data.trip)]
check

In [ ]:
# SV
list_ok = check.trip[(check.res_sampling_gps == 1) & (check.gap_sampling_tdr == 0) & 
                     (check.nb_dive > 0) & (check.stop < 600)]
data = data[data.trip.isin(list_ok)]
data

In [ ]:
# ## LB
# list_ok = check.trip[(check.gap_sampling_tdr == 0) &
#                      (check.res_sampling_gps == 1) & 
#                      (check.nb_sampling_tdr > 1000) &
#                      (check.stop < 3000)]
# data = data[data.trip.isin(list_ok)]
# data

In [ ]:
# # SD
# list_ok = check.trip[(check.res_sampling_gps == 15) &(check.nb_dive > 0) & (check.res_sampling_gps <= 15)]
# data = data[data.trip.isin(list_ok)]
# data

In [ ]:
# # SS
# list_ok = check.trip[(check.res_sampling_gps == 15) & (check.longest_gap < 100) & (check.nb_fastlog_event > 5)]
# data = data[data.trip.isin(list_ok)]
# data

In [ ]:
trip_duration = []
trip_dives = []

In [ ]:
for i in data.trip.unique():
    t = data[data.trip == i]
    
    # compute dives
    trip = Trip(t)
    trip.add_dive(2)
    
#     trip.df['dive'] = 1*trip.df['fastlog']
    
    trip_duration.append(trip.get_duration().total_seconds()/60)
    trip_dives.append(sum(trip.df.dive))
    
    # fill dive in main table
    data = data.copy()
    data.loc[data.trip == i, 'dive'] = trip.df.dive.values
    
#     # plot individual and global plots
#     trip.plot('./data/SS/'+ str(i)+'.png')
#     plt.plot(np.array(t.lon), np.array(t.lat))
    
print('Total nb of trips: '+ str(len(data.trip.unique())))

In [ ]:
sum(data.dive)/len(data)

In [ ]:
np.std(trip_dives)

In [ ]:
# data.to_csv('./data/SS_all.csv', index = False)

## split to train, validation, test dataset

In [ ]:
data_guanape = data[data.trip.str[:1]=='G']
data = data[data.trip.str[:1]=='P']

In [ ]:
data_guanape.to_csv('./data/SV_test_guanape.csv', index = False)

In [ ]:
trajs = data.trip.unique()
np.random.shuffle(trajs)
trajs

In [ ]:
nb_train = round(0.7 * len(data.trip.unique()))
nb_validation = round(0.2 * len(data.trip.unique()))
nb_test = round(0.1 * len(data.trip.unique()))

In [ ]:
train_trajs = trajs[0:nb_train]
validation_trajs = trajs[nb_train:(nb_train+nb_validation)]
test_trajs = trajs[(nb_train+nb_validation):(nb_train+nb_validation+nb_test)]

In [ ]:
data_train = data[data.trip.isin(train_trajs)]
data_validation = data[data.trip.isin(validation_trajs)]
data_test = data[data.trip.isin(test_trajs)]

In [ ]:
data_train.to_csv('./data/SV_train.csv', index = False)
data_validation.to_csv('./data/SV_validation.csv', index = False)
data_test.to_csv('./data/SV_test.csv', index = False)

In [ ]:
(nb_train, nb_validation, nb_test)